In [188]:
import pandas as pd
import numpy as np
import os,random

In [189]:
seed=42
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(seed)

In [190]:
train=pd.read_csv('/content/drive/MyDrive/ulianov/train_exif.csv')
test=pd.read_csv('/content/drive/MyDrive/ulianov/test_exif.csv')

In [191]:
'''train['x_min2']=train['x_min']-train['x_min_car']
train['y_min2']=train['y_min']-train['y_min_car']
train['x_max2']=train['x_max']-train['x_min_car']
train['y_max2']=train['y_max']-train['y_min_car']

test['x_min2']=test['x_min']-test['x_min_car']
test['y_min2']=test['y_min']-test['y_min_car']
test['x_max2']=test['x_max']-test['x_min_car']
test['y_max2']=test['y_max']-test['y_min_car']'''

"train['x_min2']=train['x_min']-train['x_min_car']\ntrain['y_min2']=train['y_min']-train['y_min_car']\ntrain['x_max2']=train['x_max']-train['x_min_car']\ntrain['y_max2']=train['y_max']-train['y_min_car']\n\ntest['x_min2']=test['x_min']-test['x_min_car']\ntest['y_min2']=test['y_min']-test['y_min_car']\ntest['x_max2']=test['x_max']-test['x_min_car']\ntest['y_max2']=test['y_max']-test['y_min_car']"

In [192]:
#train=train.drop(columns=['x_min_car','y_min_car','x_max_car','y_max_car','conf_car','class_car'])
#test=test.drop(columns=['x_min_car','y_min_car','x_max_car','y_max_car','conf_car','class_car'])

In [193]:
'''temp_fit_df=pd.concat((train,test))

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
fit_data=np.array(temp_fit_df['x_min']).reshape(-1,1)
scaler.fit(fit_data)
transform_data=np.array(train['x_min']).reshape(-1,1)
train['x_min']=scaler.transform(transform_data)
transform_data=np.array(test['x_min']).reshape(-1,1)
test['x_min']=scaler.transform(transform_data)

scaler = StandardScaler()
fit_data=np.array(temp_fit_df['y_min']).reshape(-1,1)
scaler.fit(fit_data)
transform_data=np.array(train['y_min']).reshape(-1,1)
train['y_min']=scaler.transform(transform_data)
transform_data=np.array(test['y_min']).reshape(-1,1)
test['y_min']=scaler.transform(transform_data)

scaler = StandardScaler()
fit_data=np.array(temp_fit_df['x_max']).reshape(-1,1)
scaler.fit(fit_data)
transform_data=np.array(train['x_max']).reshape(-1,1)
train['x_max']=scaler.transform(transform_data)
transform_data=np.array(test['x_max']).reshape(-1,1)
test['x_max']=scaler.transform(transform_data)

scaler = StandardScaler()
fit_data=np.array(temp_fit_df['y_max']).reshape(-1,1)
scaler.fit(fit_data)
transform_data=np.array(train['y_max']).reshape(-1,1)
train['y_max']=scaler.transform(transform_data)
transform_data=np.array(test['y_max']).reshape(-1,1)
test['y_max']=scaler.transform(transform_data)'''

"temp_fit_df=pd.concat((train,test))\n\nfrom sklearn.preprocessing import StandardScaler\nscaler = StandardScaler()\nfit_data=np.array(temp_fit_df['x_min']).reshape(-1,1)\nscaler.fit(fit_data)\ntransform_data=np.array(train['x_min']).reshape(-1,1)\ntrain['x_min']=scaler.transform(transform_data)\ntransform_data=np.array(test['x_min']).reshape(-1,1)\ntest['x_min']=scaler.transform(transform_data)\n\nscaler = StandardScaler()\nfit_data=np.array(temp_fit_df['y_min']).reshape(-1,1)\nscaler.fit(fit_data)\ntransform_data=np.array(train['y_min']).reshape(-1,1)\ntrain['y_min']=scaler.transform(transform_data)\ntransform_data=np.array(test['y_min']).reshape(-1,1)\ntest['y_min']=scaler.transform(transform_data)\n\nscaler = StandardScaler()\nfit_data=np.array(temp_fit_df['x_max']).reshape(-1,1)\nscaler.fit(fit_data)\ntransform_data=np.array(train['x_max']).reshape(-1,1)\ntrain['x_max']=scaler.transform(transform_data)\ntransform_data=np.array(test['x_max']).reshape(-1,1)\ntest['x_max']=scaler.tra

In [194]:
print(train.isna().sum())

image_name    0
distance      0
x_min         6
y_min         6
x_max         6
             ..
lens3         0
subjcloc1     0
subjcloc2     0
subjcloc3     0
subjcloc4     0
Length: 75, dtype: int64


In [195]:
train['width']=train['x_max']-train['x_min']
train['height']=train['y_max']-train['y_min']

test['width']=test['x_max']-test['x_min']
test['height']=test['y_max']-test['y_min']

In [196]:
train['width']=1/train['width']
train['height']=1/train['width']

test['width']=1/test['width']
test['height']=1/test['width']

In [197]:
drops=['image_name','class']

train=train.drop(columns=drops)



In [198]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [199]:
pd.set_option('display.max_columns', 500)
from catboost import CatBoostRegressor

In [200]:
#print(train['OffsetTimeDigitized'].value_counts())

In [201]:


def prepare(x):
    x = x.split(' ')
    date = x[0].split(':')
    time = x[1].split(':')
    year,month,day = map(int, date)
    timestamp = pd.Timestamp(f'{day}-{month}-{year}')
    hour = int(time[0])
    dayofyear = timestamp.dayofyear
    dayofweek = timestamp.dayofweek
    if dayofweek >= 5:
        days_to_weekend = 0
    else:
        days_to_weekend = min(dayofweek + 1, abs(5 - dayofweek))
    return year, month, day, hour, dayofyear, dayofweek, dayofweek>=5, (month-1) // 3, days_to_weekend

add_columns = ['year', 'month', 'day', 'hour', 'dayofyear', 'dayofweek', 'is_weekend', \
               'season', 'days_to_weekend']
train[add_columns] = list(train['DateTime'].apply(prepare))
for col in add_columns:
    train[col] = train[col].astype(np.int32)
  
test[add_columns] = list(test['DateTime'].apply(prepare))
for col in add_columns:
    test[col] = test[col].astype(np.int32)

for i,val in enumerate(add_columns):
  add_columns[i]=val+'2'

train[add_columns] = list(train['DateTimeOriginal'].apply(prepare))
for col in add_columns:
    train[col] = train[col].astype(np.int32)
  
test[add_columns] = list(test['DateTimeOriginal'].apply(prepare))
for col in add_columns:
    test[col] = test[col].astype(np.int32)

for i,val in enumerate(add_columns):
  add_columns[i]=val+'3'

train[add_columns] = list(train['DateTimeDigitized'].apply(prepare))
for col in add_columns:
    train[col] = train[col].astype(np.int32)
  
test[add_columns] = list(test['DateTimeDigitized'].apply(prepare))
for col in add_columns:
    test[col] = test[col].astype(np.int32)

date_drops=['DateTimeOriginal','DateTimeDigitized','DateTime']
train=train.drop(columns=date_drops)
test=test.drop(columns=date_drops)

In [202]:
for i in train.columns:#                     
  print(train[i].value_counts()/len(train))
  print()

full_disbalanced=['ResolutionUnit','Make','XResolution','YResolution','ExifVersion','Flash','ColorSpace','ExifImageWidth','OffsetTime','OffsetTimeOriginal','OffsetTimeDigitized','ExifImageHeight','SensingMethod','SceneType','ExposureProgram','ExposureMode','WhiteBalance','LensMake','1','2','3','4','5','12','16','23','21_gps','41_gps','year','month','is_weekend','season','year2','month2','is_weekend2','season2','year23','month23','is_weekend23','season23']
#full_disbalanced=['ResolutionUnit','XResolution','YResolution','Flash','ColorSpace','ExifImageWidth','ExifImageHeight','SensingMethod','ExposureProgram','ExposureMode','WhiteBalance','2','4','21_gps','41_gps','year','month','is_weekend','season','1_N','3_E','12_K','16_T','23_T','Make_Apple',"ExifVersion_b'0232'",'OffsetTime_+04:00','OffsetTimeOriginal_+04:00','OffsetTimeDigitized_+04:00',"SceneType_b'\x01'",'LensMake_Apple',"5_b'\x00'"]
partially_disbalanced=['ApertureValue','Orientation','ExposureBiasValue','MeteringMode','FNumber','CompositeImage','13','lens1','lens2','lens3']

2.10    0.009434
4.42    0.007547
4.41    0.007547
3.24    0.007547
3.52    0.007547
          ...   
6.00    0.001887
4.17    0.001887
7.55    0.001887
5.09    0.001887
5.20    0.001887
Name: distance, Length: 348, dtype: float64

219.604050    0.001887
103.592194    0.001887
149.841110    0.001887
255.470154    0.001887
177.300095    0.001887
                ...   
194.177078    0.001887
267.287781    0.001887
131.098465    0.001887
140.876999    0.001887
181.400436    0.001887
Name: x_min, Length: 524, dtype: float64

402.193756    0.001887
204.923050    0.001887
314.474548    0.001887
702.439575    0.001887
412.821014    0.001887
                ...   
236.720749    0.001887
355.527618    0.001887
231.912506    0.001887
260.147736    0.001887
356.746887    0.001887
Name: y_min, Length: 524, dtype: float64

425.297180    0.001887
238.980759    0.001887
352.669220    0.001887
695.618835    0.001887
440.045044    0.001887
                ...   
406.994904    0.001887
594.195923    0.0

In [203]:
print(list(train.columns))

['distance', 'x_min', 'y_min', 'x_max', 'y_max', 'conf', 'ResolutionUnit', 'ExifOffset', 'Make', 'Model', 'Software', 'Orientation', 'XResolution', 'YResolution', 'HostComputer', 'ExifVersion', 'ShutterSpeedValue', 'ApertureValue', 'BrightnessValue', 'ExposureBiasValue', 'MeteringMode', 'Flash', 'FocalLength', 'ColorSpace', 'ExifImageWidth', 'DigitalZoomRatio', 'FocalLengthIn35mmFilm', 'OffsetTime', 'OffsetTimeOriginal', 'OffsetTimeDigitized', 'SubsecTimeOriginal', 'SubsecTimeDigitized', 'ExifImageHeight', 'SensingMethod', 'ExposureTime', 'FNumber', 'SceneType', 'ExposureProgram', 'ISOSpeedRatings', 'ExposureMode', 'WhiteBalance', 'LensMake', 'LensModel', 'CompositeImage', '1', '2', '3', '4', '5', '6', '12', '13', '16', '17', '23', '24', '31', '21_gps', '22_gps', '23_gps', '41_gps', '42_gps', '43_gps', 'lens1', 'lens2', 'lens3', 'subjcloc1', 'subjcloc2', 'subjcloc3', 'subjcloc4', 'width', 'height', 'year', 'month', 'day', 'hour', 'dayofyear', 'dayofweek', 'is_weekend', 'season', 'days_

In [204]:
train=train.drop(columns=full_disbalanced)
test=test.drop(columns=full_disbalanced)#partially_disbalanced

train=train.drop(columns=partially_disbalanced)
test=test.drop(columns=partially_disbalanced)

In [205]:
print(train)

     distance       x_min        y_min        x_max        y_max      conf  \
0        4.88  219.604050   402.193756   425.297180   455.929901  0.842233   
1        1.54  614.961609  1326.627197  1302.240479  1479.057617  0.864385   
2        3.68  298.759644   461.289703   591.384766   529.101013  0.865617   
3        2.22  501.177734   748.439636   983.731384   856.448853  0.874792   
4        3.73  294.875153   471.319153   591.095154   539.254272  0.868262   
..        ...         ...          ...          ...          ...       ...   
525      3.00  306.027252   650.916504   604.249329   720.184814  0.865165   
526      3.46  269.867737   575.216919   522.212463   635.551270  0.865655   
527      3.91  249.696381   501.747467   478.498352   556.968689  0.857690   
528      5.20  198.687332   382.187531   368.827576   425.248566  0.837795   
529      5.54  181.400436   356.746887   344.161957   398.403748  0.829634   

     ExifOffset      Model Software HostComputer  ShutterSpeedV

In [206]:
print(train['LensModel'].unique())

['iPhone 11 back dual wide camera 1.54mm f/2.4'
 'iPhone 11 back dual wide camera 4.25mm f/1.8'
 'iPhone 12 back dual wide camera 1.55mm f/2.4']


In [207]:
train['camera_mm']=1.54
train.loc[train['LensModel']=='iPhone 11 back dual wide camera 4.25mm f/1.8','camera_mm']=4.25
train.loc[train['LensModel']=='iPhone 12 back dual wide camera 1.55mm f/2.4','camera_mm']=1.55

train['camera_f/']=2.4
train.loc[train['LensModel']=='iPhone 11 back dual wide camera 4.25mm f/1.8','camera_f/']=1.8

test['camera_mm']=1.54
test.loc[test['LensModel']=='iPhone 11 back dual wide camera 4.25mm f/1.8','camera_mm']=4.25
test.loc[test['LensModel']=='iPhone 12 back dual wide camera 1.55mm f/2.4','camera_mm']=1.55

test['camera_f/']=2.4
test.loc[test['LensModel']=='iPhone 11 back dual wide camera 4.25mm f/1.8','camera_f/']=1.8

In [208]:
'''train['bbox_area']=train['height']*train['width']
test['bbox_area']=test['height']*test['width']'''

"train['bbox_area']=train['height']*train['width']\ntest['bbox_area']=test['height']*test['width']"

In [209]:
train['tl_br']=np.sqrt(((train['x_min']-train['x_max'])**2)+((train['y_min']-train['y_max'])**2))
test['tl_br']=np.sqrt(((test['x_min']-test['x_max'])**2)+((test['y_min']-test['y_max'])**2))

In [210]:
print(train['camera_mm'].value_counts())

1.54    497
1.55     27
4.25      6
Name: camera_mm, dtype: int64


In [211]:
print(list(train.columns))

['distance', 'x_min', 'y_min', 'x_max', 'y_max', 'conf', 'ExifOffset', 'Model', 'Software', 'HostComputer', 'ShutterSpeedValue', 'BrightnessValue', 'FocalLength', 'DigitalZoomRatio', 'FocalLengthIn35mmFilm', 'SubsecTimeOriginal', 'SubsecTimeDigitized', 'ExposureTime', 'ISOSpeedRatings', 'LensModel', '6', '17', '24', '31', '22_gps', '23_gps', '42_gps', '43_gps', 'subjcloc1', 'subjcloc2', 'subjcloc3', 'subjcloc4', 'width', 'height', 'day', 'hour', 'dayofyear', 'dayofweek', 'days_to_weekend', 'day2', 'hour2', 'dayofyear2', 'dayofweek2', 'days_to_weekend2', 'day23', 'hour23', 'dayofyear23', 'dayofweek23', 'days_to_weekend23', 'camera_mm', 'camera_f/', 'tl_br']


In [212]:
for i in train.columns:#                     
  print(train[i].value_counts()/len(train))
  print()

2.10    0.009434
4.42    0.007547
4.41    0.007547
3.24    0.007547
3.52    0.007547
          ...   
6.00    0.001887
4.17    0.001887
7.55    0.001887
5.09    0.001887
5.20    0.001887
Name: distance, Length: 348, dtype: float64

219.604050    0.001887
103.592194    0.001887
149.841110    0.001887
255.470154    0.001887
177.300095    0.001887
                ...   
194.177078    0.001887
267.287781    0.001887
131.098465    0.001887
140.876999    0.001887
181.400436    0.001887
Name: x_min, Length: 524, dtype: float64

402.193756    0.001887
204.923050    0.001887
314.474548    0.001887
702.439575    0.001887
412.821014    0.001887
                ...   
236.720749    0.001887
355.527618    0.001887
231.912506    0.001887
260.147736    0.001887
356.746887    0.001887
Name: y_min, Length: 524, dtype: float64

425.297180    0.001887
238.980759    0.001887
352.669220    0.001887
695.618835    0.001887
440.045044    0.001887
                ...   
406.994904    0.001887
594.195923    0.0

In [213]:
#print(train)
drops=['Make','Model','DateTime','HostComputer','ExifVersion','DateTimeOriginal',
       'DateTimeDigitized','OffsetTime','OffsetTimeOriginal','OffsetTimeDigitized','SceneType',
       'LensMake','5']

#train=pd.get_dummies(train,columns=['LensModel','1','3','12','16','23','Software'])
#test=pd.get_dummies(test,columns=['LensModel','1','3','12','16','23','Software'])

train=pd.get_dummies(train,columns=['LensModel','Software'])
test=pd.get_dummies(test,columns=['LensModel','Software'])

'''train=pd.get_dummies(train,columns=['Make','Model','HostComputer','ExifVersion',
                                    'OffsetTime','OffsetTimeOriginal','OffsetTimeDigitized',
                                    'SceneType','LensMake','5'])
test=pd.get_dummies(test,columns=['Make','Model','HostComputer','ExifVersion',
                                    'OffsetTime','OffsetTimeOriginal','OffsetTimeDigitized',
                                    'SceneType','LensMake','5'])'''

train=pd.get_dummies(train,columns=['Model','HostComputer'])
test=pd.get_dummies(test,columns=['Model','HostComputer'])
#train=train.drop(columns=drops)
#test=test.drop(columns=drops)
print(train)

     distance       x_min        y_min        x_max        y_max      conf  \
0        4.88  219.604050   402.193756   425.297180   455.929901  0.842233   
1        1.54  614.961609  1326.627197  1302.240479  1479.057617  0.864385   
2        3.68  298.759644   461.289703   591.384766   529.101013  0.865617   
3        2.22  501.177734   748.439636   983.731384   856.448853  0.874792   
4        3.73  294.875153   471.319153   591.095154   539.254272  0.868262   
..        ...         ...          ...          ...          ...       ...   
525      3.00  306.027252   650.916504   604.249329   720.184814  0.865165   
526      3.46  269.867737   575.216919   522.212463   635.551270  0.865655   
527      3.91  249.696381   501.747467   478.498352   556.968689  0.857690   
528      5.20  198.687332   382.187531   368.827576   425.248566  0.837795   
529      5.54  181.400436   356.746887   344.161957   398.403748  0.829634   

     ExifOffset  ShutterSpeedValue  BrightnessValue  FocalLengt

In [214]:
from tqdm import tqdm
import gc

In [215]:
'''custom_data=[]
for i in tqdm(range(len(train))):
    for j in range(39,2250,40): #X
        for z in range(105,1785,40): #Y
            cur_data=dict(train.loc[i])
            cur_data['x_min']=j
            cur_data['x_max']=j+cur_data['width']
            cur_data['y_min']=z
            cur_data['y_max']=z+cur_data['height']
            custom_data.append(cur_data)
temp_df=pd.DataFrame(custom_data)
del custom_data
gc.collect()
train=pd.concat((train,temp_df))
train.to_csv('/content/drive/MyDrive/ulianov/custom_exif_new_train.csv')'''

"custom_data=[]\nfor i in tqdm(range(len(train))):\n    for j in range(39,2250,40): #X\n        for z in range(105,1785,40): #Y\n            cur_data=dict(train.loc[i])\n            cur_data['x_min']=j\n            cur_data['x_max']=j+cur_data['width']\n            cur_data['y_min']=z\n            cur_data['y_max']=z+cur_data['height']\n            custom_data.append(cur_data)\ntemp_df=pd.DataFrame(custom_data)\ndel custom_data\ngc.collect()\ntrain=pd.concat((train,temp_df))\ntrain.to_csv('/content/drive/MyDrive/ulianov/custom_exif_new_train.csv')"

In [216]:
#train=train.dropna()

X=train.drop('distance',axis=1)
y=train['distance']


In [217]:
#X=X.drop(columns=['x_min','y_min','x_max','y_max'])
#test=test.drop(columns=['x_min','y_min','x_max','y_max'])

In [218]:
unimportant=[
'ExifOffset',
'ShutterSpeedValue',
'BrightnessValue',
'FocalLength',
'SubsecTimeOriginal',
'SubsecTimeDigitized',
'ExposureTime',
'ISOSpeedRatings',
'6',
'22_gps',
'42_gps',
'43_gps',
'subjcloc1',
'subjcloc2',
'camera_mm',
'camera_f/',
'LensModel_iPhone 11 back dual wide camera 1.54mm f/2.4',
'LensModel_iPhone 12 back dual wide camera 1.55mm f/2.4',
'Software_15.4.1',
'Software_15.5',
'Model_iPhone 11',
'Model_iPhone 12',
'HostComputer_iPhone 11',
'HostComputer_iPhone 12',
]

#X=X.drop(columns=unimportant)
#est=test.drop(columns=unimportant)

In [219]:
#print(test_vals.isna().sum())

In [220]:
from sklearn.model_selection import train_test_split

In [221]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=seed)

In [222]:
import matplotlib.pyplot as plt

In [223]:
'''
cb=CatBoostRegressor(iterations=50000,random_state=seed,task_type='GPU',learning_rate=0.001,random_strength=2,eval_metric='R2')
cb.fit(X_train,y_train,verbose=1000,eval_set=(X_val,y_val))

plt.bar(X_train.columns,cb.get_feature_importance())
plt.show()
temp_df=pd.DataFrame([X_train.columns,cb.get_feature_importance()])
print(temp_df)'''

from sklearn.ensemble import RandomForestRegressor

cb=CatBoostRegressor(iterations=20000,random_state=seed,task_type='CPU',learning_rate=0.001,random_strength=2)
cb.fit(X_train,y_train,verbose=1000)



#rfc=RandomForestRegressor(n_estimators=9000,random_state=seed,verbose=100)
#rfc.fit(X,y)

val_preds=cb.predict(X_val)
#val_preds=rfc.predict(X_val)

val_d={
    'preds':val_preds,
    'true':y_val
}

val_df=pd.DataFrame(val_d)
val_df['diff']=val_df['true']-val_df['preds']

val_df.to_csv('/content/drive/MyDrive/ulianov/val_true_diff.csv',index=False)

test_vals=test.drop(columns=['image_name','class'])
preds=cb.predict(test_vals)
#test_vals['DigitalZoomRatio']=test_vals['DigitalZoomRatio'].fillna(1.022312)
#preds=rfc.predict(test_vals)

sub_d={
    'image_name':list(test[['image_name']].values),
    'distance':preds
}

sub=pd.DataFrame(sub_d)
sub['image_name']=list(map(lambda x:x[0],sub['image_name']))
print(sub)
sub.to_csv('/content/drive/MyDrive/ulianov/res.csv',index=False,sep=';')

0:	learn: 1.7324146	total: 52.3ms	remaining: 17m 26s
1000:	learn: 0.9486969	total: 4.26s	remaining: 1m 20s
2000:	learn: 0.6051261	total: 16.7s	remaining: 2m 30s
3000:	learn: 0.4490943	total: 30.4s	remaining: 2m 52s
4000:	learn: 0.3702705	total: 34.3s	remaining: 2m 17s
5000:	learn: 0.3241369	total: 38.2s	remaining: 1m 54s
6000:	learn: 0.2919381	total: 42s	remaining: 1m 38s
7000:	learn: 0.2668356	total: 45.9s	remaining: 1m 25s
8000:	learn: 0.2456161	total: 49.8s	remaining: 1m 14s
9000:	learn: 0.2288746	total: 53.8s	remaining: 1m 5s
10000:	learn: 0.2133001	total: 57.7s	remaining: 57.7s
11000:	learn: 0.1980559	total: 1m 1s	remaining: 50.5s
12000:	learn: 0.1832486	total: 1m 5s	remaining: 43.8s
13000:	learn: 0.1699569	total: 1m 9s	remaining: 37.5s
14000:	learn: 0.1589528	total: 1m 13s	remaining: 31.5s
15000:	learn: 0.1487982	total: 1m 17s	remaining: 25.8s
16000:	learn: 0.1398631	total: 1m 21s	remaining: 20.4s
17000:	learn: 0.1321086	total: 1m 25s	remaining: 15.1s
18000:	learn: 0.1245861	tota

In [224]:
print('[')
for i,val in enumerate(cb.get_feature_importance()):
  if val<0.1:
    print(f"'{X_train.columns[i]}',")
print(']')

[
'ExifOffset',
'FocalLength',
'camera_mm',
'camera_f/',
'LensModel_iPhone 11 back dual wide camera 1.54mm f/2.4',
'LensModel_iPhone 11 back dual wide camera 4.25mm f/1.8',
'LensModel_iPhone 12 back dual wide camera 1.55mm f/2.4',
'Software_15.4.1',
'Software_15.5',
'Model_iPhone 11',
'Model_iPhone 12',
'HostComputer_iPhone 11',
'HostComputer_iPhone 12',
]
